In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection, metrics  #to include metrics for evaluation # this used to be cross_validation
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import matplotlib.pylab as plt

In [5]:
df_telco = pd.read_csv('../../model_testing/UnBalancedData/Churn Prediction/Telco-Customer-Churn.csv', usecols=['gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'])
df_telco.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [6]:
data = df_telco.copy()
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors = 'coerce')
data[data['TotalCharges'].isna()==True] = 0

data['gender'].replace(['Male','Female'],[0,1],inplace=True)
data['Partner'].replace(['Yes','No'],[1,0],inplace=True)
data['Dependents'].replace(['Yes','No'],[1,0],inplace=True)
data['PhoneService'].replace(['Yes','No'],[1,0],inplace=True)
data['MultipleLines'].replace(['No phone service','No', 'Yes'],[0,0,1],inplace=True)
data['InternetService'].replace(['No','DSL','Fiber optic'],[0,1,2],inplace=True)
data['OnlineSecurity'].replace(['No','Yes','No internet service'],[0,1,0],inplace=True)
data['OnlineBackup'].replace(['No','Yes','No internet service'],[0,1,0],inplace=True)
data['DeviceProtection'].replace(['No','Yes','No internet service'],[0,1,0],inplace=True)
data['TechSupport'].replace(['No','Yes','No internet service'],[0,1,0],inplace=True)
data['StreamingTV'].replace(['No','Yes','No internet service'],[0,1,0],inplace=True)
data['StreamingMovies'].replace(['No','Yes','No internet service'],[0,1,0],inplace=True)
data['Contract'].replace(['Month-to-month', 'One year', 'Two year'],[0,1,2],inplace=True)
data['PaperlessBilling'].replace(['Yes','No'],[1,0],inplace=True)
data['PaymentMethod'].replace(['Electronic check', 'Mailed check', 'Bank transfer (automatic)','Credit card (automatic)'],[0,1,2,3],inplace=True)
data['Churn'].replace(['Yes','No'],[1,0],inplace=True)

data.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,1,0,1,0,1,0,0,1,0,1,0,0,0,0,0,1,0,29.85,29.85,0
1,0,0,0,0,34,1,0,1,1,0,1,0,0,0,1,0,1,56.95,1889.50,0
2,0,0,0,0,2,1,0,1,1,1,0,0,0,0,0,1,1,53.85,108.15,1
3,0,0,0,0,45,0,0,1,1,0,1,1,0,0,1,0,2,42.30,1840.75,0
4,1,0,0,0,2,1,0,2,0,0,0,0,0,0,0,1,0,70.70,151.65,1


## Freature Reduction

### With RFE

In [7]:
X = data.drop('Churn', axis=1)
y = data.Churn

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_new = scaler.fit_transform(X)

In [9]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.feature_selection import RFE

rfr = RandomForestClassifier(random_state=42)
rfe = RFE(estimator=rfr, step=1)

In [10]:
rfe = rfe.fit(X_new, y)

In [11]:
X_rfe = X.loc[:, rfe.support_]
X_new2 = scaler.fit_transform(X_rfe)

In [12]:
# X_rfe.columns

# Build a classification model using various methods

In [13]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [14]:
model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto',max_iter=-1),
        'params' : {
            'C': [1,10,20],
#             'kernel': ['linear', 'rbf',],
            #'shrinking':['True', 'False']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10,100],
            'max_depth': [1,2,3,4,5,6],
            'min_samples_leaf': [100,200,300,500,1000]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(multi_class='auto'),
        'params': {
            'C': [1,5,10],
            'solver':['newton-cg', 'liblinear']
        }
    },
    'naive_bayes_gaussian': {
        'model': GaussianNB(),
        'params': {}
    },
    'kneighbours': {
        'model': KNeighborsClassifier(n_jobs=-1),
        "params": {
            
            'n_neighbors' : [5, 10 , 100],
        }
    },
    'Xgboost': {
        'model': XGBClassifier(n_jobs=-1),
        "params": {
            'n_estimators': [100, 115, 150],
            'max_depth': [3,4,5,6, 10],
#                    'max_features': [None, 'auto', 'sqrt', 'log2'], 
#                    'min_samples_split': [2, 3, 4, 5, 10]
           
        }
    },
    'LightGBM': {
        'model': LGBMClassifier(n_jobs=-1),
        "params": {
            'n_estimators': [1,5,10,100],
            'max_depth': [1,2,3,4,5,6],
            'num_leaves': [100,200,300,500,1000]
        }
    },

}

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(n_splits=10)
scores = []

for model_name, mp in model_params.items():
    print(model_name)
    clf =  GridSearchCV(mp['model'], mp['params'], cv=cv, return_train_score=False)
    clf.fit(X_new2, y)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_.round(4),
        'best_params': clf.best_params_
    })

In [16]:
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df.sort_values('best_score', ascending=False)

,model,best_score,best_params
1,random_forest,0.7995,"{'max_depth': 6, 'min_samples_leaf': 100, 'n_e..."
0,svm,0.7994,{'C': 20}
6,LightGBM,0.7985,"{'max_depth': 4, 'n_estimators': 100, 'num_lea..."
2,logistic_regression,0.7984,"{'C': 10, 'solver': 'newton-cg'}"
5,Xgboost,0.7934,"{'max_depth': 3, 'n_estimators': 100}"
4,kneighbours,0.7924,{'n_neighbors': 100}
3,naive_bayes_gaussian,0.7481,{}


# Creating Pipeline

In [17]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [18]:
df_telco = pd.read_csv('../../model_testing/UnBalancedData/Churn Prediction/Telco-Customer-Churn.csv', 
                       usecols=['gender', 'Partner', 'tenure', 'InternetService', 'Contract', 'PaperlessBilling',
                                'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'])

df_telco['TotalCharges'] = pd.to_numeric(df_telco['TotalCharges'], errors = 'coerce')
df_telco['TotalCharges'].fillna(0, inplace=True)

df_telco.head()

,gender,Partner,tenure,InternetService,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,Yes,1,DSL,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,No,34,DSL,One year,No,Mailed check,56.95,1889.50,No
2,Male,No,2,DSL,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,No,45,DSL,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,No,2,Fiber optic,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [19]:
X = df_telco.drop('Churn', axis=1)
y = df_telco[['Churn']]


numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(X)
categorical_columns = categorical_columns_selector(X)

In [47]:
for i in categorical_columns:
    print(i, X[i].unique())

gender ['Female' 'Male']
Partner ['Yes' 'No']
InternetService ['DSL' 'Fiber optic' 'No']
Contract ['Month-to-month' 'One year' 'Two year']
PaperlessBilling ['Yes' 'No']
PaymentMethod ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']


In [22]:
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()

In [23]:
preprocessor = ColumnTransformer(
    [
        ("one-hot-encoder", categorical_preprocessor, categorical_columns),
        ("standard_scaler", numerical_preprocessor, numerical_columns),
    ]
)

In [24]:
model = make_pipeline(preprocessor, RandomForestClassifier(**df.loc[df['model']=='random_forest', 'best_params'].item()))
model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('one-hot-encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['gender', 'Partner',
                                                   'InternetService',
                                                   'Contract',
                                                   'PaperlessBilling',
                                                   'PaymentMethod']),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  ['tenure', 'MonthlyCharges',
                                                   'TotalCharges'])])),
                ('randomforestclassifier',
                 RandomForestClassifier(max_depth=6, min_samples_leaf=100))])

In [42]:
data_train, data_test, target_train, target_test = train_test_split(X, y, stratify=y, random_state=42)

In [43]:
_ = model.fit(data_train, target_train)

C:\Users\Utkarsh.Goyal\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [44]:
model.predict(data_test)[:5]

array(['No', 'No', 'No', 'Yes', 'No'], dtype=object)

In [50]:
data_test

,gender,Partner,tenure,InternetService,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
5909,Female,No,52,Fiber optic,One year,Yes,Mailed check,80.20,4297.60
3670,Female,Yes,33,No,Month-to-month,No,Bank transfer (automatic),24.25,838.50
6220,Male,Yes,10,DSL,Two year,No,Credit card (automatic),53.70,521.00
5905,Male,No,1,Fiber optic,Month-to-month,Yes,Electronic check,85.00,85.00
6435,Male,Yes,52,DSL,Two year,No,Mailed check,50.20,2554.00
...,...,...,...,...,...,...,...,...,...
476,Male,Yes,35,DSL,Month-to-month,Yes,Electronic check,76.05,2747.20
1607,Male,Yes,13,Fiber optic,Month-to-month,Yes,Bank transfer (automatic),89.05,1169.35
6808,Male,No,39,Fiber optic,Two year,Yes,Electronic check,100.45,3801.70
2962,Female,Yes,3,DSL,Month-to-month,Yes,Mailed check,31.00,95.05


# target_test[:5]

In [46]:
model.score(data_test, target_test)

0.7910278250993753

In [49]:
import pickle

with open("model.pkl", "wb") as f:
    pickle.dump(model, f)